In [1]:
import numpy as np

np.random.seed(0)

In [2]:
max_bits = 10
n_samples = 100000

In [3]:
samples = np.random.randint(np.power(2, max_bits-1), size=(n_samples, 2))
summed = np.sum(samples, axis=1)

In [4]:
samples_binary_repr = [[np.binary_repr(a, width=max_bits), np.binary_repr(b, width=max_bits)] for a,b in samples]
summed_binary_repr = [np.binary_repr(c, width=max_bits) for c in summed]


In [5]:
x_str = np.array([[list(a), list(b)] for a, b in samples_binary_repr])
y_str = np.array([list(c) for c in summed_binary_repr])

In [6]:
x_flipped = np.flip(x_str, axis=-1)
y_flipped = np.flip(y_str, axis=-1)

In [7]:
x = np.transpose((x_flipped == '1')*1, axes=(0, 2, 1))
y = (y_flipped == '1')*1

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics, models

tf.random.set_seed(1)

In [10]:
class FullAdderCell(layers.Layer):
    def __init__(self, hidden_units, **kwargs):
        self.units = 1
        self.state_size = 1
        self.hidden_units = hidden_units
        super(FullAdderCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.hidden_weights = self.add_weight(shape=(input_shape[-1] + self.state_size, self.hidden_units),
                                      initializer='uniform',
                                      name='hidden_weights')
        self.hidden_bias = self.add_weight(shape=(1, self.hidden_units),
                                      initializer='uniform',
                                      name='hidden_bias')
        self.output_weights = self.add_weight(shape=(self.hidden_units, self.units + self.state_size),
                                      initializer='uniform',
                                      name='output_weights')
        self.output_bias = self.add_weight(shape=(1, self.units + self.state_size),
                                      initializer='uniform',
                                      name='output_bias')
        self.built = True

    def call(self, inputs, states):
        x = tf.concat([inputs, states[0]], axis=-1)
        h = tf.keras.activations.tanh(tf.matmul(x, self.hidden_weights) + self.hidden_bias)
        o_s = tf.keras.activations.sigmoid(tf.matmul(h, self.output_weights) + self.output_bias)
        output = o_s[:, :self.units]
        state = o_s[:, self.units:]
        return output, [state]


In [11]:
model = tf.keras.Sequential()
model.add(layers.RNN(FullAdderCell(3), return_sequences=True, input_shape=(None, 2)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, None, 1)           20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=5)
scores = model.evaluate(x_test, y_test, verbose=2)
print(scores)

Train on 90000 samples
Epoch 1/5
90000/90000 [==============================] - 12s 135us/sample - loss: 0.6929 - accuracy: 0.5039
Epoch 2/5
90000/90000 [==============================] - 11s 121us/sample - loss: 0.5155 - accuracy: 0.7632
Epoch 3/5
90000/90000 [==============================] - 11s 124us/sample - loss: 0.0609 - accuracy: 1.0000
Epoch 4/5
90000/90000 [==============================] - 10s 110us/sample - loss: 0.0126 - accuracy: 1.0000
Epoch 5/5
90000/90000 [==============================] - 10s 113us/sample - loss: 0.0040 - accuracy: 1.0000
10000/1 - 1s - loss: 0.0022 - accuracy: 1.0000
[0.0021821730833500625, 1.0]
